### You want to be in the '../International-Maritime-Piracy/notebooks' folder

In [1]:
import os
import re

import pandas as pd

In [2]:
data_directory = os.path.join('../data','RAW_all_piracy_events.csv')

df = pd.read_csv(data_directory)
df.head()

,reference,date,position,navArea,subreg,hostility,victim,description,Unnamed: 8
0,2022-127,2022-04-28,"13°43'00""N \n121°02'00""E",XI,91,NaN,Marshall Islands-flagged tanker SEXTANS,"PHILIPPINES: On 28 April, at 0345 local time, ...",NaN
1,2022-124,2022-04-27,"1°16'00""N \n104°17'00""E",XI,71,Boarding,Bulk carrier,"INDONESIA On 27 April, at 0226 local time, a r...",NaN
2,2022-121,2022-04-26,"0°04'00""S \n51°06'00""W",V,24,Boarding,Bulk carrier,"BRAZIL: On 26 April, at 0400 local time, three...",NaN
3,2022-128,2022-04-26,"22°13'00""N \n91°43'00""E",VIII,63,NaN,Singapore-flagged tanker STI MAGISTER,"BANGLADESH: On 26 April, at 0001 local time, a...",NaN
4,2022-122,2022-04-25,"59°19'00""N \n10°30'00""E",I,44,Protest,Hong Kong-flagged tanker UST LUGA,"NORWAY: On the morning of 25 April, a group of...",NaN


In [3]:
df.drop(['Unnamed: 8'], axis=1, inplace=True)

In [4]:
df.head()

,reference,date,position,navArea,subreg,hostility,victim,description
0,2022-127,2022-04-28,"13°43'00""N \n121°02'00""E",XI,91,NaN,Marshall Islands-flagged tanker SEXTANS,"PHILIPPINES: On 28 April, at 0345 local time, ..."
1,2022-124,2022-04-27,"1°16'00""N \n104°17'00""E",XI,71,Boarding,Bulk carrier,"INDONESIA On 27 April, at 0226 local time, a r..."
2,2022-121,2022-04-26,"0°04'00""S \n51°06'00""W",V,24,Boarding,Bulk carrier,"BRAZIL: On 26 April, at 0400 local time, three..."
3,2022-128,2022-04-26,"22°13'00""N \n91°43'00""E",VIII,63,NaN,Singapore-flagged tanker STI MAGISTER,"BANGLADESH: On 26 April, at 0001 local time, a..."
4,2022-122,2022-04-25,"59°19'00""N \n10°30'00""E",I,44,Protest,Hong Kong-flagged tanker UST LUGA,"NORWAY: On the morning of 25 April, a group of..."


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8619 entries, 0 to 8618
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   reference    8619 non-null   object
 1   date         8619 non-null   object
 2   position     8619 non-null   object
 3   navArea      8616 non-null   object
 4   subreg       8619 non-null   int64 
 5   hostility    8107 non-null   object
 6   victim       8327 non-null   object
 7   description  8613 non-null   object
dtypes: int64(1), object(7)
memory usage: 538.8+ KB


### Split postion into Lat/Lon in prep for conversion to decimal degrees

In [6]:
df = df.join(df['position'].str.split(expand=True)).rename(columns={0:'latitude', 1:'longitude'})
df.drop('position', axis=1, inplace=True)

In [7]:
df.head(1)

,reference,date,navArea,subreg,hostility,victim,description,latitude,longitude
0,2022-127,2022-04-28,XI,91,NaN,Marshall Islands-flagged tanker SEXTANS,"PHILIPPINES: On 28 April, at 0345 local time, ...","13°43'00""N","121°02'00""E"


In [8]:
def dms2dd(s):
    # example: s = """0°51'56.29"S"""
    degrees, minutes, seconds, direction = re.split('[°\'"]+', s)
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
    if direction in ('S','W'):
        dd*= -1
    return dd

In [9]:
df['latitude'] = df['latitude'].apply(dms2dd).round(6)
df['longitude'] = df['longitude'].apply(dms2dd).round(6)
df.head(4)

,reference,date,navArea,subreg,hostility,victim,description,latitude,longitude
0,2022-127,2022-04-28,XI,91,NaN,Marshall Islands-flagged tanker SEXTANS,"PHILIPPINES: On 28 April, at 0345 local time, ...",13.716667,121.033333
1,2022-124,2022-04-27,XI,71,Boarding,Bulk carrier,"INDONESIA On 27 April, at 0226 local time, a r...",1.266667,104.283333
2,2022-121,2022-04-26,V,24,Boarding,Bulk carrier,"BRAZIL: On 26 April, at 0400 local time, three...",-0.066667,-51.100000
3,2022-128,2022-04-26,VIII,63,NaN,Singapore-flagged tanker STI MAGISTER,"BANGLADESH: On 26 April, at 0001 local time, a...",22.216667,91.716667


### What's up with the 'hostility' column?

In [10]:
df['hostility'].value_counts()[100:150]

ETHIOPIA                      2
Three Pirates                 2
LTTE TERRORISTS               2
SUSPICOUS APPROACH            2
TWO SPEED BOATS               2
PRIATES                       2
UNIDENTIFIED ATTACKERS        2
CHINESE AUTHORITIES           2
POSSIBLE TERRORISTS           2
FIVE PIRATES                  2
FOUR PIRATES                  2
FISHING BOATS                 2
ATTEMPTED BOARDING            2
MILITANTS                     2
PIRACY                        2
Two white speedboats          1
PRATES                        1
SRI LANKAN NAVY               1
NAVAL PERSONNEL               1
PIRATES ATTACK                1
MILITIAMEN                    1
SHORESIDE PIRATES             1
PIRATES / KIDNAPPING          1
SUSPICUIOUS APPROACH          1
SUSPECTED PIRATE              1
Terrorist Attack              1
SUS[ICIOUS APPROACH           1
YACHT                         1
SEPARIST REBELS               1
suspicous approach            1
SUSPIPICIOUS APPROACH         1
IMMIGRAN

### Under 'hostility' - how many different terms are there?

In [11]:
df['hostility'].nunique()

431

### Under 'victim' - how many different terms are there?

In [12]:
df['victim'].nunique()

1589

In [13]:
df['victim'].value_counts()[900:950]

GS FABULOUS                                           1
AP 703                                                1
Container vessel BAHMAS                               1
AMINA                                                 1
BERTHED CHEMICAL TANKER                               1
SEA DIAMOND                                           1
ANCHORED GENERAL CARGO SHIP                           1
MOHAMED BEY                                           1
Liberia-flagged container ship ELA                    1
ARGUELLO                                              1
WATER PHOENIX                                         1
BOW ORION                                             1
QUEEN OF THE NETHERLANDS                              1
Underway oil tanker PIKE                              1
ANCHORED TANKER                                       1
BLUE COMMANDER                                        1
RIO CHARLESTON                                        1
HAI LU FENG 11                                  

### Convert datetime to just the year for a slider

In [14]:
df['date2'] = pd.to_datetime(df['date'], format='%Y/%m/%d')
df.head(1)

,reference,date,navArea,subreg,hostility,victim,description,latitude,longitude,date2
0,2022-127,2022-04-28,XI,91,NaN,Marshall Islands-flagged tanker SEXTANS,"PHILIPPINES: On 28 April, at 0345 local time, ...",13.716667,121.033333,2022-04-28


In [15]:
df['year'] = pd.DatetimeIndex(df['date2']).year
df["date"] = pd.to_datetime(df["date2"])

df = df[['reference', 'latitude', 'longitude', 'date', 'year', 'description', 'hostility', 'victim', 'navArea', 'subreg' ]]

# sanity check
df.head()

,reference,latitude,longitude,date,year,description,hostility,victim,navArea,subreg
0,2022-127,13.716667,121.033333,2022-04-28,2022,"PHILIPPINES: On 28 April, at 0345 local time, ...",NaN,Marshall Islands-flagged tanker SEXTANS,XI,91
1,2022-124,1.266667,104.283333,2022-04-27,2022,"INDONESIA On 27 April, at 0226 local time, a r...",Boarding,Bulk carrier,XI,71
2,2022-121,-0.066667,-51.100000,2022-04-26,2022,"BRAZIL: On 26 April, at 0400 local time, three...",Boarding,Bulk carrier,V,24
3,2022-128,22.216667,91.716667,2022-04-26,2022,"BANGLADESH: On 26 April, at 0001 local time, a...",NaN,Singapore-flagged tanker STI MAGISTER,VIII,63
4,2022-122,59.316667,10.500000,2022-04-25,2022,"NORWAY: On the morning of 25 April, a group of...",Protest,Hong Kong-flagged tanker UST LUGA,I,44


### Right, the NaNs would mess things up. Let's convert them to something that can be parsed later

In [16]:
df.fillna("No information", inplace=True)
df.isnull().sum().sum()
df.head(5)

,reference,latitude,longitude,date,year,description,hostility,victim,navArea,subreg
0,2022-127,13.716667,121.033333,2022-04-28,2022,"PHILIPPINES: On 28 April, at 0345 local time, ...",No information,Marshall Islands-flagged tanker SEXTANS,XI,91
1,2022-124,1.266667,104.283333,2022-04-27,2022,"INDONESIA On 27 April, at 0226 local time, a r...",Boarding,Bulk carrier,XI,71
2,2022-121,-0.066667,-51.100000,2022-04-26,2022,"BRAZIL: On 26 April, at 0400 local time, three...",Boarding,Bulk carrier,V,24
3,2022-128,22.216667,91.716667,2022-04-26,2022,"BANGLADESH: On 26 April, at 0001 local time, a...",No information,Singapore-flagged tanker STI MAGISTER,VIII,63
4,2022-122,59.316667,10.500000,2022-04-25,2022,"NORWAY: On the morning of 25 April, a group of...",Protest,Hong Kong-flagged tanker UST LUGA,I,44


In [17]:
os.makedirs('../data/in_progress', exist_ok=True)  

df.to_csv('../data/in_progress/v1_lat_long.csv') 

# Future work

1. remove the excess spaces in the description column.

2. hostility - easy first pass would be to trust the data ruthlessly pare down the columns

3. victim - easy first pass would be to trust the data ruthlessly pare down the columns

4. description - NLP to set the hostility and possibly victim columns


#### The excel file for the description has a lot a weird spacing. Does that show here?

Why, yes it does.

In [18]:
df['description'][df["reference"]=='1991-21']

8450    W. MEDITERRANIAN-COAST OF ALGERIA-VICINITY CAP...
Name: description, dtype: object

### Let's regularize the 'hostility' columns values

In [19]:
first_cut = df.hostility.nunique()
first_cut

432

In [20]:
h = df.hostility.unique()
h.sort()
h

array([' SUSPICIOUS APPROACH', '15 PIRATES', '24-APR-2002',
       '25 SPEEDBOATS', 'ACTIVISTS', 'ALGERIAN COASTAL PATROL',
       'ALLEGED POLISARIO FRONT', 'AMERBER II', 'ARMED PIRATES',
       'ARMED ROBBERS', 'ARMED ROBBERY', 'ASSAILANTS', 'ATTACKER',
       'ATTACKERS', 'ATTEMPTED BOARDING', 'AUTHORITIES IN JACMEL, HAITI',
       'Algerian Patrol Vessel',
       'Approached by fishing boat with four armed men',
       'Approached by skiff', 'Armed Pirates', 'Armed boarders',
       'Armed men in a speedboat', 'Armed persons boarding',
       'Armed persons in a skiff fired upon a tanker',
       'Armed robbers boarded vessel', 'Armed robbers boarding',
       'Armed robebrs', 'Assailants', 'Attacked and boarded',
       'Attacked and fired upon', 'Attacked three passenger boats',
       'Attempted Boarding', 'Attempted boarding', 'Attempted robbery',
       'BANDITS', 'BLUE SPEEDBOAT', 'BOARDED BY ARMED ROBBERS', 'BOARDER',
       'BOARDERS', 'BOARDING', 'BOAT', 'BURMESE PIRATES',

In [21]:
#df[df["hostility"].str.contains(r"^Su(s|c)(p|])", regex=True, case=False)]


df['hostility'] = df['hostility'].str.replace(r'(?i)^Su[sc][][p][ie].*', 'Suspicious Approach', regex=True)

In [22]:
second_cut = df.hostility.nunique()
print(f" This took us from {first_cut} to {second_cut}")

 This took us from 432 to 399


In [23]:
df['hostility'] = df['hostility'].str.replace(r'(?i)Protest.*', 'Protest', regex=True)
df['hostility'] = df['hostility'].str.replace(r'(?i)Stow.*', 'Stowaway', regex=True)

In [24]:
third_cut = df.hostility.nunique()
print(f" This took us from {second_cut} to {third_cut}")

 This took us from 399 to 394
